In [6]:
!pip install scikit-learn==1.3.0 --user
!pip install matplotlib



DEPRECATION: pytorch-lightning 1.7.7 has a non-standard dependency specifier torch>=1.9.*. pip 24.0 will enforce this behaviour change. A possible replacement is to upgrade to a newer version of pytorch-lightning or contact the author to suggest that they release a version with a conforming dependency specifiers. Discussion can be found at https://github.com/pypa/pip/issues/12063


DEPRECATION: pytorch-lightning 1.7.7 has a non-standard dependency specifier torch>=1.9.*. pip 24.0 will enforce this behaviour change. A possible replacement is to upgrade to a newer version of pytorch-lightning or contact the author to suggest that they release a version with a conforming dependency specifiers. Discussion can be found at https://github.com/pypa/pip/issues/12063


In [8]:
import pandas as pd
import numpy as np

import seaborn as sb
import time
import datetime
from sklearn.datasets import make_classification
from sklearn.model_selection import train_test_split
from matplotlib.backend_bases import time
from pandas._libs.tslibs.offsets import Day
# Importar las bibliotecas necesarias
import tkinter as tk
from tkinter import ttk
from matplotlib.figure import Figure
from matplotlib.backends.backend_tkagg import FigureCanvasTkAgg
import random
import joblib
from sklearn.metrics import roc_curve, roc_auc_score
import pandas as pd
import tkinter.filedialog
from collections import Counter




# Cargar el modelo desde el archivo
fraud_probabilities = []
# Una lista para guardar las predicciones
predictions_list = []

model = joblib.load('C:/Users/carlo/fraud_detection_model.pkl')

def load_csv():
    filename = tkinter.filedialog.askopenfilename(filetypes=[("Archivos CSV", "*.csv")])
    if not filename:  # Si el usuario cancela la operación
        return
    data = pd.read_csv(filename)
    batch_predict(data)
    
def batch_predict(data):
    # Asegúrate de que las columnas del CSV coincidan con las que necesita tu modelo
    predictions = model.predict(data)
    probas = model.predict_proba(data)[:, 1]  # Probabilidades de ser fraude

    # Actualizar la lista global de probabilidades de fraude y predicciones
    fraud_probabilities.extend(probas)
    predictions_list.extend([int(x) for x in predictions])

    # Actualizar los gráficos
    update_probability_distribution_plot()
    update_prediction_distribution_plot()


def update_prediction_distribution_plot():
    fig = Figure(figsize=(5, 4), dpi=100)
    ax = fig.add_subplot(111)
    
    # Contar la frecuencia de cada tipo de predicción
    prediction_counts = Counter(predictions_list)
    
    labels = list(prediction_counts.keys())
    counts = list(prediction_counts.values())
    
    ax.bar(labels, counts, color='green')
    ax.set_title('Distribución de Predicciones')
    ax.set_xlabel('Predicción')
    ax.set_ylabel('Frecuencia')
    ax.set_xticks([0, 1])  # Establecer las marcas del eje X para que solo muestre 0 y 1
    
    canvas = FigureCanvasTkAgg(fig, master=window)
    canvas.draw()
    canvas.get_tk_widget().grid(row=14, column=1) 


def predict_and_metrics(input_df):
    # Realizar la predicción usando el modelo
    prediction = model.predict(input_df)
    prediction_proba = model.predict_proba(input_df)[:, 1]
    
    return prediction, prediction_proba
# Simula una función de predicción para el ejemplo
def predict(amt, city_pop, lat, long, category, state, day, time, month, weekday):
    
    input_dict = {
        'amt': [float(amt)],
        'city_pop': [int(city_pop)],
        'lat': [float(lat)],
        'long': [float(long)],
        'category': [category],
        'state': [state],
        'trans_time_rounded':[time],
        'month': [month],  # Nueva variable
        'day': [day],  # Nueva variable
        'weekday': [weekday]  # Nueva variable
    }
       # Realizar la predicción usando el modelo
    input_df = pd.DataFrame.from_dict(input_dict)
    prediction = model.predict(input_df)
    prediction_prob = model.predict_proba(input_df)
    fraud_prob = prediction_prob[0][1]
    return prediction, fraud_prob


# Función que se ejecutará cuando se presione el botón "Predecir"
def button_predict():
    amt = amt_entry.get()
    city_pop = city_pop_entry.get()
    lat = lat_entry.get()
    long = long_entry.get()
    category = category_var.get()
    state = edo_entry.get()
    time = hora_entry.get()
    month = month_entry.get()
    day = day_entry.get()
    weekday = weekday_entry.get()
    
    prediction, fraud_prob = predict(amt, city_pop, lat, long, category, state, day, time, month, weekday)
    fraud_probabilities.append(fraud_prob)
        # Guardar la predicción en la lista
    predictions_list.append(int(prediction[0]))
    
    # Actualizar los gráficos
    update_probability_distribution_plot()
    update_prediction_distribution_plot()
    
    # Actualizar el gráfico
    update_probability_distribution_plot() 
    prediction_text = "Predicción: Fraudulento" if prediction == 1 else "Predicción: No Fraudulento"
    
    result_window = tk.Toplevel(window)
    result_window.title("Resultado de la Predicción")
    ttk.Label(result_window, text=prediction_text, font=("Helvetica", 16)).pack(padx=20, pady=10)
    ttk.Label(result_window, text=f"Probabilidad de fraude: {fraud_prob:.2f}", font=("Helvetica", 16)).pack(padx=20, pady=10)
    ttk.Button(result_window, text="Cerrar", command=result_window.destroy).pack(pady=10)

def update_probability_distribution_plot():
    fig = Figure(figsize=(5, 4), dpi=100)
    ax = fig.add_subplot(111)
    ax.hist(fraud_probabilities, bins=20, color='blue', edgecolor='black')
    ax.set_title('Distribución de Probabilidades de Fraude')
    ax.set_xlabel('Probabilidad')
    ax.set_ylabel('Frecuencia')

    canvas = FigureCanvasTkAgg(fig, master=window)
    canvas.draw()
    canvas.get_tk_widget().grid(row=14, column=0)  # Cambio aquí: columna 0


def clear_entries():
    amt_entry.delete(0, 'end')
    city_pop_entry.delete(0, 'end')
    lat_entry.delete(0, 'end')
    long_entry.delete(0, 'end')
    month_entry.delete(0, 'end')
    day_entry.delete(0, 'end')
    weekday_entry.delete(0, 'end')
    category_var.delete(0, 'end')
    edo_entry.delete(0, 'end')
    hora_entry.delete(0, 'end')



    
# Iniciar la ventana de Tkinter
window = tk.Tk()
window.title("Predicción de Fraude")

title = tk.Label(window, text="Detección de Fraude", font=("Helvetica", 24))
title.grid(row=0, columnspan=2, pady=20)

# Aplicar estilos
style = ttk.Style()
style.theme_use('clam')
style.configure('TButton', background='#368', foreground='white')
style.map('TButton', background=[('active', '#4a9')])
style.configure('TEntry', fieldbackground='#aaa', foreground='#000')
style.configure('TLabel', background='#fff', foreground='#000')
style.configure('TMenubutton', background='#aaa', foreground='#000')

# Fuentes
customFont = ('Helvetica', 14)

# Añadir cuadros de texto para variables numéricas
ttk.Label(window, text="Cantidad:", font=customFont).grid(row=1, column=0, padx=10, pady=10)
amt_entry = ttk.Entry(window, font=customFont)
amt_entry.grid(row=1, column=1, padx=10, pady=10)

ttk.Label(window, text="Población de la Ciudad:", font=customFont).grid(row=2, column=0, padx=10, pady=10)
city_pop_entry = ttk.Entry(window, font=customFont)
city_pop_entry.grid(row=2, column=1, padx=10, pady=10)

ttk.Label(window, text="Latitud:", font=customFont).grid(row=3, column=0, padx=10, pady=10)
lat_entry = ttk.Entry(window, font=customFont)
lat_entry.grid(row=3, column=1, padx=10, pady=10)

ttk.Label(window, text="Longitud:", font=customFont).grid(row=4, column=0, padx=10, pady=10)
long_entry = ttk.Entry(window, font=customFont)
long_entry.grid(row=4, column=1, padx=10, pady=10)


ttk.Label(window, text="Hora:", font=customFont).grid(row=5, column=0, padx=10, pady=10)
hora_entry = ttk.Entry(window, font=customFont)
hora_entry.grid(row=5, column=1, padx=10, pady=10)

# Añadir cuadros de texto para las nuevas variables
ttk.Label(window, text="Mes:", font=customFont).grid(row=6, column=0, padx=10, pady=10)
month_entry = ttk.Entry(window, font=customFont)
month_entry.grid(row=6, column=1, padx=10, pady=10)

ttk.Label(window, text="Día:", font=customFont).grid(row=7, column=0, padx=10, pady=10)
day_entry = ttk.Entry(window, font=customFont)
day_entry.grid(row=7, column=1, padx=10, pady=10)

ttk.Label(window, text="Día de la semana:", font=customFont).grid(row=8, column=0, padx=10, pady=10)
weekday_entry = ttk.Entry(window, font=customFont)
weekday_entry.grid(row=8, column=1, padx=10, pady=10)


ttk.Label(window, text="Estado:", font=customFont).grid(row=9, column=0, padx=10, pady=10)
edo_entry = ttk.Entry(window, font=customFont)  # Nombre de variable ajustado
edo_entry.grid(row=9, column=1, padx=10, pady=10)

ttk.Label(window, text="Categoría:", font=customFont).grid(row=10, column=0, padx=10, pady=10)
category_var = ttk.Entry(window, font=customFont)
category_var.grid(row=10, column=1, padx=10, pady=10)
# Botón para ejecutar la predicción
ttk.Button(window, text="Predecir", command=button_predict).grid(row=11,  column=0, pady=20)
ttk.Button(window, text="Limpiar", command=clear_entries).grid(row=11, column=1,  columnspan=2, pady=20)
ttk.Button(window, text="Cargar CSV", command=load_csv).grid(row=16, columnspan=2, pady=20)

# Mantener la ventana abierta
window.mainloop()


ModuleNotFoundError: No module named 'seaborn'